In [ ]:
import xarray as xr
import numpy as np
from scipy.stats import wilcoxon
import netCDF4 as nc
import pandas as pd


# Define parameters for analysis
start_year = 2014
end_year = 2015
longhurst_region_code = 38
region_name = "NPTG"
combined_cat_values = [3, 4]  # Magnitude of heatwave categories to consider
num_samples = 100  
consecutive_months_threshold = 1

# Section 1: Load and Preprocess Data
# Load the netCDF file containing variables other than chlorophyll
dataset = xr.open_dataset('/Users/sayooj/Downloads/GlobalAtlas_MHW_ESACCISST_1deg_1982-2021.nc', decode_times=False)

# Define the start and end indices for slicing
start_idx = (start_year - 1982) * 365
end_idx = start_idx + (end_year - start_year + 1) * 365 - 1

# Create a new dataset with data only for the specified time range
new_dataset = dataset.isel(time=slice(start_idx, end_idx + 1))

# Convert data variables to float32 if needed
new_dataset['cat'] = new_dataset['cat'].astype('float32')
new_dataset['mhw'] = new_dataset['mhw'].astype('float32')

# Save the new dataset to a new netCDF file
new_dataset.to_netcdf(f'/Users/sayooj/Downloads/{region_name}_{start_year}_{end_year}.nc')

# Section 2: Mask Based on Longhurst Regions
# Open the Longhurst region file
longhurst_file = '/Users/sayooj/Downloads/Longhurst_1_deg.nc'
longhurst_dataset = xr.open_dataset(longhurst_file)

# Read the Longhurst variable
longhurst = longhurst_dataset['longhurst'].values

# Create a mask based on Longhurst regions and transpose it
mask = np.isin(longhurst, [longhurst_region_code]).T

# Apply the mask to the entire time range
masked_dataset = new_dataset.where(mask)

# Save the masked data to a new netCDF file
masked_file_path = f'/Users/sayooj/Downloads/masked_{region_name}_{start_year}_{end_year}.nc'
masked_dataset.to_netcdf(masked_file_path)

# Section 3: Create Monthly Masks with Values Only Inside Longhurst Region
# Load the netCDF file containing the masked data
masked_nc_file = xr.open_dataset(masked_file_path, decode_times=False)

# Extract the masked cat variable and apply the Longhurst mask
masked_cat = masked_nc_file['cat'].where(mask)

# Calculate the number of months
num_months = int(len(masked_nc_file['time']) / 30)

# Create an empty array to store monthly masks
monthly_masks = np.zeros((num_months, len(masked_nc_file['lat']), len(masked_nc_file['lon']))) * np.nan

# Iterate over each month
for month in range(num_months):
    # Calculate the start and end indices for the current month
    start_idx = month * 30
    end_idx = (month + 1) * 30

    # Extract the masked daily cat values for the current month
    month_data = masked_cat[start_idx:end_idx]

    # Find the maximum category occurrence for each lat-lon point in the current month
    max_values = np.max(month_data, axis=0)

    # Set areas impacted by the highest category occurrence within the Longhurst region
    monthly_mask = np.where(mask, max_values, np.nan)

    # Save the monthly mask
    monthly_masks[month] = monthly_mask

# Create a new netCDF file to save the monthly masks
output_file = xr.Dataset(
    data_vars={
        'lat': ('lat', masked_nc_file['lat'].values),
        'lon': ('lon', masked_nc_file['lon'].values),
        'time': ('time', np.arange(1, num_months + 1)),
        'monthly_masks': (['time', 'lat', 'lon'], monthly_masks)
    }
)

# Add attributes
output_file['lat'].attrs['units'] = 'degrees_north'
output_file['lon'].attrs['units'] = 'degrees_east'
output_file['time'].attrs['units'] = f'months since {start_year}-01-01'
output_file['monthly_masks'].attrs['units'] = '1'
output_file.attrs['description'] = f'Monthly masks for marine heatwaves in {region_name}'

# Save the monthly masks to a new netCDF file
output_file.to_netcdf(f'/Users/sayooj/Downloads/monthly_masks_{region_name}_{start_year}_{end_year}.nc')

# Close the netCDF files
masked_nc_file.close()

# Section 4: Create Consecutive Monthly Mask for Values 3 or 4
# Load the netCDF file containing the monthly masks
monthly_masks_file = xr.open_dataset(f'/Users/sayooj/Downloads/monthly_masks_{region_name}_{start_year}_{end_year}.nc', decode_times=False)

# Extract the monthly masks variable
monthly_masks_data = monthly_masks_file['monthly_masks'].values

# Initialize the consecutive monthly mask array
consecutive_monthly_mask = np.zeros_like(monthly_masks) * np.nan

# Iterate over each lat-lon point
for lat_idx in range(monthly_masks_data.shape[1]):
    for lon_idx in range(monthly_masks_data.shape[2]):
        # Extract the monthly mask values for the current lat-lon point
        values = monthly_masks_data[:, lat_idx, lon_idx]

        consecutive_count = 0
        consecutive_mask = np.zeros_like(values)

        for i in range(len(values)):
            if (values[i] == 3) or (values[i] == 4):
                consecutive_count += 1
                consecutive_mask[i] = values[i]
            else:
                consecutive_count = 0
                consecutive_mask[i] = 0

            if consecutive_count >= consecutive_months_threshold:
                break

        # Set the consecutive monthly mask values for the current lat-lon point
        consecutive_monthly_mask[:len(consecutive_mask), lat_idx, lon_idx] = consecutive_mask

# Apply the Longhurst mask to set values inside the region to NaN
consecutive_monthly_mask = np.where(mask, consecutive_monthly_mask, np.nan)



# Create a new netCDF file to save the consecutive monthly mask
consecutive_monthly_mask_file = xr.Dataset(
    data_vars={
        'lat': ('lat', monthly_masks_file['lat'].values),
        'lon': ('lon', monthly_masks_file['lon'].values),
        'time': ('time', monthly_masks_file['time'].values),
        'consecutive_monthly_mask': (['time', 'lat', 'lon'], consecutive_monthly_mask)
    }
)

# Add attributes
consecutive_monthly_mask_file['lat'].attrs['units'] = 'degrees_north'
consecutive_monthly_mask_file['lon'].attrs['units'] = 'degrees_east'
consecutive_monthly_mask_file['time'].attrs['units'] = f'months since {start_year}-01-01'
consecutive_monthly_mask_file['consecutive_monthly_mask'].attrs['units'] = '1'
consecutive_monthly_mask_file.attrs['description'] = f'Consecutive monthly mask for values 3 or 4 in {region_name}'

# Save the consecutive monthly mask to a new netCDF file
consecutive_monthly_mask_file.to_netcdf(f'/Users/sayooj/Downloads/consecutive_monthly_mask_{region_name}_{start_year}_{end_year}.nc')

# Close the netCDF files
monthly_masks_file.close()
consecutive_monthly_mask_file.close()

# Initialize a list to store information about consecutive heatwaves
consecutive_heatwave_info = []

# Iterate over each lat-lon point
for lat_idx in range(consecutive_monthly_mask.shape[1]):
    for lon_idx in range(consecutive_monthly_mask.shape[2]):
        # Extract the consecutive monthly mask values for the current lat-lon point
        values = consecutive_monthly_mask[:, lat_idx, lon_idx]

        # Find indices where consecutive heatwaves occurred (values 3 or 4)
        heatwave_indices = np.where(np.isin(values, [3, 4]))[0]

        # If consecutive heatwaves occurred at this lat-lon point
        if len(heatwave_indices) >= consecutive_months_threshold:
            # Get the corresponding dates for the identified indices
            heatwave_dates = monthly_masks_file['time'].values[heatwave_indices]

            # Convert months to dates based on the start_year
            start_date = pd.to_datetime(f'{start_year}-01-01')
            exact_dates = [(start_date + pd.DateOffset(months=int(month))).strftime('%Y-%m-%d') for month in heatwave_dates]

            # Append the lat, lon, months, and exact_dates to the list
            consecutive_heatwave_info.append({
                'lat': monthly_masks_file['lat'].values[lat_idx],
                'lon': monthly_masks_file['lon'].values[lon_idx],
                'months': heatwave_dates.tolist(),
                'exact_dates': exact_dates
            })

# Create a DataFrame with the extracted information
consecutive_heatwave_info_df = pd.DataFrame(consecutive_heatwave_info)

# Save the DataFrame to a CSV file with region name, start year, and end year in the filename
csv_file_path = f'/Users/sayooj/Downloads/consecutive_heatwave_info_{region_name}_{start_year}_{end_year}.csv'
consecutive_heatwave_info_df.to_csv(csv_file_path, index=False)


# Section 4: Create Consecutive Monthly Mask for Values 3 or 4
# Load the netCDF file containing the monthly masks
monthly_masks_file = xr.open_dataset(f'/Users/sayooj/Downloads/monthly_masks_{region_name}_{start_year}_{end_year}.nc', decode_times=False)

# Extract the monthly masks variable
monthly_masks_data = monthly_masks_file['monthly_masks'].values

# Initialize the consecutive monthly mask array
consecutive_monthly_mask = np.zeros_like(monthly_masks) * np.nan

# Iterate over each lat-lon point
for lat_idx in range(monthly_masks_data.shape[1]):
    for lon_idx in range(monthly_masks_data.shape[2]):
        # Extract the monthly mask values for the current lat-lon point
        values = monthly_masks_data[:, lat_idx, lon_idx]

        consecutive_count = 0
        consecutive_mask = np.zeros_like(values)

        for i in range(len(values)):
            if (values[i] == 3) or (values[i] == 4):
                consecutive_count += 1
                consecutive_mask[i] = values[i]
            else:
                consecutive_count = 0
                consecutive_mask[i] = 0

            if consecutive_count >= consecutive_months_threshold:
                break

        # Set the consecutive monthly mask values for the current lat-lon point
        consecutive_monthly_mask[:len(consecutive_mask), lat_idx, lon_idx] = consecutive_mask

# Apply the Longhurst mask to set values inside the region to NaN
consecutive_monthly_mask = np.where(mask, consecutive_monthly_mask, np.nan)



# Create a new netCDF file to save the consecutive monthly mask
consecutive_monthly_mask_file = xr.Dataset(
    data_vars={
        'lat': ('lat', monthly_masks_file['lat'].values),
        'lon': ('lon', monthly_masks_file['lon'].values),
        'time': ('time', monthly_masks_file['time'].values),
        'consecutive_monthly_mask': (['time', 'lat', 'lon'], consecutive_monthly_mask)
    }
)

# Add attributes
consecutive_monthly_mask_file['lat'].attrs['units'] = 'degrees_north'
consecutive_monthly_mask_file['lon'].attrs['units'] = 'degrees_east'
consecutive_monthly_mask_file['time'].attrs['units'] = f'months since {start_year}-01-01'
consecutive_monthly_mask_file['consecutive_monthly_mask'].attrs['units'] = '1'
consecutive_monthly_mask_file.attrs['description'] = f'Consecutive monthly mask for values 3 or 4 in {region_name}'

# Save the consecutive monthly mask to a new netCDF file
consecutive_monthly_mask_file.to_netcdf(f'/Users/sayooj/Downloads/consecutive_monthly_mask_{region_name}_{start_year}_{end_year}.nc')

# Close the netCDF files
monthly_masks_file.close()
consecutive_monthly_mask_file.close()

# Initialize a list to store information about consecutive heatwaves
consecutive_heatwave_info = []

# Iterate over each lat-lon point
for lat_idx in range(consecutive_monthly_mask.shape[1]):
    for lon_idx in range(consecutive_monthly_mask.shape[2]):
        # Extract the consecutive monthly mask values for the current lat-lon point
        values = consecutive_monthly_mask[:, lat_idx, lon_idx]

        # Find indices where consecutive heatwaves occurred (values 3 or 4)
        heatwave_indices = np.where(np.isin(values, [3, 4]))[0]

        # If consecutive heatwaves occurred at this lat-lon point
        if len(heatwave_indices) >= consecutive_months_threshold:
            # Get the corresponding dates for the identified indices
            heatwave_dates = monthly_masks_file['time'].values[heatwave_indices]

            # Convert months to dates based on the start_year
            start_date = pd.to_datetime(f'{start_year}-01-01')
            exact_dates = [(start_date + pd.DateOffset(months=int(month))).strftime('%Y-%m-%d') for month in heatwave_dates]

            # Append the lat, lon, months, and exact_dates to the list
            consecutive_heatwave_info.append({
                'lat': monthly_masks_file['lat'].values[lat_idx],
                'lon': monthly_masks_file['lon'].values[lon_idx],
                'months': heatwave_dates.tolist(),
                'exact_dates': exact_dates
            })

# Create a DataFrame with the extracted information
consecutive_heatwave_info_df = pd.DataFrame(consecutive_heatwave_info)

# Save the DataFrame to a CSV file with region name, start year, and end year in the filename
csv_file_path = f'/Users/sayooj/Downloads/consecutive_heatwave_info_{region_name}_{start_year}_{end_year}.csv'
consecutive_heatwave_info_df.to_csv(csv_file_path, index=False)

# Section 4: Statistical Analysis
# Open the heatwave dataset file
heatwave_dataset_file = nc.Dataset('/Users/sayooj/Downloads/OceanSODA-ETHZ_GRaCER_v2021a_1982-2020.nc')

# Open the non-heatwave dataset file
non_heatwave_dataset_file = nc.Dataset('/Users/sayooj/Downloads/all_variables_monthly_avg_overall.nc')

# Get the variable data
fgco2 = heatwave_dataset_file.variables['fgco2'][:]
ph = heatwave_dataset_file.variables['ph_total'][:]
aragonite_saturation = heatwave_dataset_file.variables['omega_ar'][:]
sst = heatwave_dataset_file.variables['temperature'][:]

# Create the time axis for the specified years
dates = pd.date_range(start='{}-01-01'.format(start_year), end='{}-12-31'.format(end_year), freq='M')

# Find the indices corresponding to the time period
indices_year = np.where(dates.year.isin([start_year, end_year]))[0]

# Slice the data for the specified years
fgco2_year = fgco2[indices_year]
sst_year = sst[indices_year]
ph_year = ph[indices_year]
aragonite_saturation_year = aragonite_saturation[indices_year]

# Open the mask file for the specified region
mask_file = nc.Dataset(f'/Users/sayooj/Downloads/consecutive_monthly_mask_{region_name}_{start_year}_{end_year}.nc')

# Get the mask variable for the specified region
mask_region = mask_file.variables['consecutive_monthly_mask'][:]

# Get indices where the mask values are equal to any of the specified cat values (e.g., heatwave period)
indices_heatwave_region_year = np.where(np.isin(mask_region, combined_cat_values))[0]

# Get indices where the mask values are not equal to any of the specified cat values (e.g., non-heatwave period)
indices_non_heatwave_region_year = np.where(~np.isin(mask_region, combined_cat_values))[0]

# Apply the mask to the sliced data
fgco2_masked_year = np.ma.masked_array(fgco2_year, np.logical_not(mask_region))
sst_masked_year = np.ma.masked_array(sst_year, np.logical_not(mask_region))
ph_masked_year = np.ma.masked_array(ph_year, np.logical_not(mask_region))
aragonite_saturation_masked_year = np.ma.masked_array(aragonite_saturation_year, np.logical_not(mask_region))

# Calculate median values with the mask for fgco2
fgco2_median_region_year = np.ma.median(fgco2_masked_year, axis=(1, 2))

# Calculate median values with the mask for sst
sst_median_region_year = np.ma.median(sst_masked_year, axis=(1, 2))

# Calculate median values with the mask for ph
ph_median_region_year = np.ma.median(ph_masked_year, axis=(1, 2))

# Calculate median values with the mask for aragonite saturation
aragonite_saturation_median_region_year = np.ma.median(aragonite_saturation_masked_year, axis=(1, 2))

# Perform the Wilcoxon signed-rank tests for fgco2
p_values_region_year = []
median_diff_region_year = []
std_dev_region_year = []

for _ in range(num_samples):
    # Randomly select indices for heatwave and non-heatwave periods
    sample_indices_region_heatwave_year = np.random.choice(indices_heatwave_region_year, len(indices_heatwave_region_year), replace=True)
    sample_indices_region_non_heatwave_year = np.random.choice(indices_non_heatwave_region_year, len(indices_heatwave_region_year), replace=True)

    # Filter the data based on the sampled indices for fgco2
    sample_fgco2_median_region_heatwave_year = fgco2_median_region_year[sample_indices_region_heatwave_year]
    sample_fgco2_median_region_non_heatwave_year = fgco2_median_region_year[sample_indices_region_non_heatwave_year]

    # Perform the Wilcoxon signed-rank tests for fgco2
    _, p_value_region_year = wilcoxon(sample_fgco2_median_region_heatwave_year, sample_fgco2_median_region_non_heatwave_year)

    # Calculate the median difference and standard deviation for fgco2
    median_diff_region_year.append(np.median(sample_fgco2_median_region_heatwave_year - sample_fgco2_median_region_non_heatwave_year))
    std_dev_region_year.append(np.std(sample_fgco2_median_region_heatwave_year - sample_fgco2_median_region_non_heatwave_year))

    # Append the p-value to the respective list for fgco2
    p_values_region_year.append(p_value_region_year)

# Calculate the median p-value, median difference, and median standard deviation for fgco2
median_p_value_fgco2_region_year = np.median(p_values_region_year)
median_median_diff_fgco2_region_year = np.median(median_diff_region_year)
median_std_dev_fgco2_region_year = np.median(std_dev_region_year)

# Perform the Wilcoxon signed-rank tests for sst
p_values_region_year = []
median_diff_region_year = []
std_dev_region_year = []

for _ in range(num_samples):
    # Randomly select indices for heatwave and non-heatwave periods
    sample_indices_region_heatwave_year = np.random.choice(indices_heatwave_region_year, len(indices_heatwave_region_year), replace=True)
    sample_indices_region_non_heatwave_year = np.random.choice(indices_non_heatwave_region_year, len(indices_heatwave_region_year), replace=True)

    # Filter the data based on the sampled indices for sst
    sample_sst_median_region_heatwave_year = sst_median_region_year[sample_indices_region_heatwave_year]
    sample_sst_median_region_non_heatwave_year = sst_median_region_year[sample_indices_region_non_heatwave_year]

    # Perform the Wilcoxon signed-rank tests for sst
    _, p_value_region_year = wilcoxon(sample_sst_median_region_heatwave_year, sample_sst_median_region_non_heatwave_year)

    # Calculate the median difference and standard deviation for sst
    median_diff_region_year.append(np.median(sample_sst_median_region_heatwave_year - sample_sst_median_region_non_heatwave_year))
    std_dev_region_year.append(np.std(sample_sst_median_region_heatwave_year - sample_sst_median_region_non_heatwave_year))

    # Append the p-value to the respective list for sst
    p_values_region_year.append(p_value_region_year)

# Calculate the median p-value, median difference, and median standard deviation for sst
median_p_value_sst_region_year = np.median(p_values_region_year)
median_median_diff_sst_region_year = np.median(median_diff_region_year)
median_std_dev_sst_region_year = np.median(std_dev_region_year)

# Perform the Wilcoxon signed-rank tests for ph
p_values_region_year = []
median_diff_region_year = []
std_dev_region_year = []

for _ in range(num_samples):
    # Randomly select indices for heatwave and non-heatwave periods
    sample_indices_region_heatwave_year = np.random.choice(indices_heatwave_region_year, len(indices_heatwave_region_year), replace=True)
    sample_indices_region_non_heatwave_year = np.random.choice(indices_non_heatwave_region_year, len(indices_heatwave_region_year), replace=True)

    # Filter the data based on the sampled indices for ph
    sample_ph_median_region_heatwave_year = ph_median_region_year[sample_indices_region_heatwave_year]
    sample_ph_median_region_non_heatwave_year = ph_median_region_year[sample_indices_region_non_heatwave_year]

    # Perform the Wilcoxon signed-rank tests for ph
    _, p_value_region_year = wilcoxon(sample_ph_median_region_heatwave_year, sample_ph_median_region_non_heatwave_year)

    # Calculate the median difference and standard deviation for ph
    median_diff_region_year.append(np.median(sample_ph_median_region_heatwave_year - sample_ph_median_region_non_heatwave_year))
    std_dev_region_year.append(np.std(sample_ph_median_region_heatwave_year - sample_ph_median_region_non_heatwave_year))

    # Append the p-value to the respective list for ph
    p_values_region_year.append(p_value_region_year)

# Calculate the median p-value, median difference, and median standard deviation for ph
median_p_value_ph_region_year = np.median(p_values_region_year)
median_median_diff_ph_region_year = np.median(median_diff_region_year)
median_std_dev_ph_region_year = np.median(std_dev_region_year)

# Perform the Wilcoxon signed-rank tests for aragonite saturation
p_values_region_year = []
median_diff_region_year = []
std_dev_region_year = []

for _ in range(num_samples):
    # Randomly select indices for heatwave and non-heatwave periods
    sample_indices_region_heatwave_year = np.random.choice(indices_heatwave_region_year, len(indices_heatwave_region_year), replace=True)
    sample_indices_region_non_heatwave_year = np.random.choice(indices_non_heatwave_region_year, len(indices_heatwave_region_year), replace=True)

    # Filter the data based on the sampled indices for aragonite saturation
    sample_aragonite_saturation_median_region_heatwave_year = aragonite_saturation_median_region_year[sample_indices_region_heatwave_year]
    sample_aragonite_saturation_median_region_non_heatwave_year = aragonite_saturation_median_region_year[sample_indices_region_non_heatwave_year]

    # Perform the Wilcoxon signed-rank tests for aragonite saturation
    _, p_value_region_year = wilcoxon(sample_aragonite_saturation_median_region_heatwave_year, sample_aragonite_saturation_median_region_non_heatwave_year)

    # Calculate the median difference and standard deviation for aragonite saturation
    median_diff_region_year.append(np.median(sample_aragonite_saturation_median_region_heatwave_year - sample_aragonite_saturation_median_region_non_heatwave_year))
    std_dev_region_year.append(np.std(sample_aragonite_saturation_median_region_heatwave_year - sample_aragonite_saturation_median_region_non_heatwave_year))

    # Append the p-value to the respective list for aragonite saturation
    p_values_region_year.append(p_value_region_year)

# Calculate the median p-value, median difference, and median standard deviation for aragonite saturation
median_p_value_aragonite_saturation_region_year = np.median(p_values_region_year)
median_median_diff_aragonite_saturation_region_year = np.median(median_diff_region_year)
median_std_dev_aragonite_saturation_region_year = np.median(std_dev_region_year)

# Define precision values for each variable (replace these with actual precision values from the paper)
precision_sst = 0.01
precision_fgco2 = 0.01 
precision_ph = 0.001 
precision_aragonite_saturation = 0.01 
precision_spco2 = 4
precision_alkalinity = 5

# Open the spco2 variable from the heatwave dataset file
spco2 = heatwave_dataset_file.variables['spco2'][:]

# Slice the spco2 data for the specified years
spco2_year = spco2[indices_year]

# Apply the mask to the sliced spco2 data
spco2_masked_year = np.ma.masked_array(spco2_year, np.logical_not(mask_region))

# Calculate median values with the mask for spco2
spco2_median_region_year = np.ma.median(spco2_masked_year, axis=(1, 2))

# Perform the Wilcoxon signed-rank tests for spco2
p_values_region_year_spco2 = []
median_diff_region_year_spco2 = []
std_dev_region_year_spco2 = []

for _ in range(num_samples):
    # Randomly select indices for heatwave and non-heatwave periods
    sample_indices_region_heatwave_year = np.random.choice(indices_heatwave_region_year, len(indices_heatwave_region_year), replace=True)
    sample_indices_region_non_heatwave_year = np.random.choice(indices_non_heatwave_region_year, len(indices_heatwave_region_year), replace=True)

    # Filter the data based on the sampled indices for spco2
    sample_spco2_median_region_heatwave_year = spco2_median_region_year[sample_indices_region_heatwave_year]
    sample_spco2_median_region_non_heatwave_year = spco2_median_region_year[sample_indices_region_non_heatwave_year]

    # Perform the Wilcoxon signed-rank tests for spco2
    _, p_value_region_year_spco2 = wilcoxon(sample_spco2_median_region_heatwave_year, sample_spco2_median_region_non_heatwave_year)

    # Calculate the median difference and standard deviation for spco2
    median_diff_region_year_spco2.append(np.median(sample_spco2_median_region_heatwave_year - sample_spco2_median_region_non_heatwave_year))
    std_dev_region_year_spco2.append(np.std(sample_spco2_median_region_heatwave_year - sample_spco2_median_region_non_heatwave_year))

    # Append the p-value to the respective list for spco2
    p_values_region_year_spco2.append(p_value_region_year_spco2)

# Calculate the median p-value, median difference, and median standard deviation for spco2
median_p_value_spco2_region_year = np.median(p_values_region_year_spco2)
median_median_diff_spco2_region_year = np.median(median_diff_region_year_spco2)
median_std_dev_spco2_region_year = np.median(std_dev_region_year_spco2)


# Get the alkalinity data
alkalinity = heatwave_dataset_file.variables['talk'][:]

# Slice the alkalinity data for the specified years
alkalinity_year = alkalinity[indices_year]

# Apply the mask to the sliced alkalinity data
alkalinity_masked_year = np.ma.masked_array(alkalinity_year, np.logical_not(mask_region))

# Calculate median values with the mask for alkalinity
alkalinity_median_region_year = np.ma.median(alkalinity_masked_year, axis=(1, 2))

# Perform the Wilcoxon signed-rank tests for alkalinity
p_values_region_year_alkalinity = []
median_diff_region_year_alkalinity = []
std_dev_region_year_alkalinity = []

for _ in range(num_samples):
    # Randomly select indices for heatwave and non-heatwave periods
    sample_indices_region_heatwave_year = np.random.choice(indices_heatwave_region_year, len(indices_heatwave_region_year), replace=True)
    sample_indices_region_non_heatwave_year = np.random.choice(indices_non_heatwave_region_year, len(indices_heatwave_region_year), replace=True)

    # Filter the data based on the sampled indices for alkalinity
    sample_alkalinity_median_region_heatwave_year = alkalinity_median_region_year[sample_indices_region_heatwave_year]
    sample_alkalinity_median_region_non_heatwave_year = alkalinity_median_region_year[sample_indices_region_non_heatwave_year]

    # Perform the Wilcoxon signed-rank tests for alkalinity
    _, p_value_region_year_alkalinity = wilcoxon(sample_alkalinity_median_region_heatwave_year, sample_alkalinity_median_region_non_heatwave_year)

    # Calculate the median difference and standard deviation for alkalinity
    median_diff_region_year_alkalinity.append(np.median(sample_alkalinity_median_region_heatwave_year - sample_alkalinity_median_region_non_heatwave_year))
    std_dev_region_year_alkalinity.append(np.std(sample_alkalinity_median_region_heatwave_year - sample_alkalinity_median_region_non_heatwave_year))

    # Append the p-value to the respective list for alkalinity
    p_values_region_year_alkalinity.append(p_value_region_year_alkalinity)

# Calculate the median p-value, median difference, and median standard deviation for alkalinity
median_p_value_alkalinity_region_year = np.median(p_values_region_year_alkalinity)
median_median_diff_alkalinity_region_year = np.median(median_diff_region_year_alkalinity)
median_std_dev_alkalinity_region_year = np.median(std_dev_region_year_alkalinity)

# Define filter functions
def apply_filter(median_diff, precision):
    return abs(median_diff) >= precision

# Apply filter for SST
if apply_filter(median_median_diff_sst_region_year, precision_sst):
    print("\nResults for SST in {} in {}/{} pass the quality filter:".format(region_name, start_year, end_year))
    print("Median p-value:", median_p_value_sst_region_year)
    print("Median median difference:", median_median_diff_sst_region_year)
    print("Median standard deviation:", median_std_dev_sst_region_year)
else:
    print("\nResults for SST in {} in {}/{} do not pass the quality filter.".format(region_name, start_year, end_year))

# Apply filter for fgco2
if apply_filter(median_median_diff_fgco2_region_year, precision_fgco2):
    print("\nResults for fgco2 in {} in {}/{} pass the quality filter:".format(region_name, start_year, end_year))
    print("Median p-value:", median_p_value_fgco2_region_year)
    print("Median median difference:", median_median_diff_fgco2_region_year)
    print("Median standard deviation:", median_std_dev_fgco2_region_year)
else:
    print("\nResults for fgco2 in {} in {}/{} do not pass the quality filter.".format(region_name, start_year, end_year))

# Apply filter for pH
if apply_filter(median_median_diff_ph_region_year, precision_ph):
    print("\nResults for pH in {} in {}/{} pass the quality filter:".format(region_name, start_year, end_year))
    print("Median p-value:", median_p_value_ph_region_year)
    print("Median median difference:", median_median_diff_ph_region_year)
    print("Median standard deviation:", median_std_dev_ph_region_year)
else:
    print("\nResults for pH in {} in {}/{} do not pass the quality filter.".format(region_name, start_year, end_year))

# Apply filter for aragonite saturation
if apply_filter(median_median_diff_aragonite_saturation_region_year, precision_aragonite_saturation):
    print("\nResults for aragonite saturation in {} in {}/{} pass the quality filter:".format(region_name, start_year, end_year))
    print("Median p-value:", median_p_value_aragonite_saturation_region_year)
    print("Median median difference:", median_median_diff_aragonite_saturation_region_year)
    print("Median standard deviation:", median_std_dev_aragonite_saturation_region_year)
else:
    print("\nResults for aragonite saturation in {} in {}/{} do not pass the quality filter.".format(region_name, start_year, end_year))

# Apply filter for spco2
if apply_filter(median_median_diff_spco2_region_year, precision_spco2):
    print("\nResults for spco2 in {} in {}/{} pass the quality filter:".format(region_name, start_year, end_year))
    print("Median p-value:", median_p_value_spco2_region_year)
    print("Median median difference:", median_median_diff_spco2_region_year)
    print("Median standard deviation:", median_std_dev_spco2_region_year)
else:
    print("\nResults for spco2 in {} in {}/{} do not pass the quality filter.".format(region_name, start_year, end_year))

# Apply filter for alkalinity
if apply_filter(median_median_diff_alkalinity_region_year, precision_alkalinity):
    print("\nResults for alkalinity in {} in {}/{} pass the quality filter:".format(region_name, start_year, end_year))
    print("Median p-value:", median_p_value_alkalinity_region_year)
    print("Median median difference:", median_median_diff_alkalinity_region_year)
    print("Median standard deviation:", median_std_dev_alkalinity_region_year)
else:
    print("\nResults for alkalinity in {} in {}/{} do not pass the quality filter.".format(region_name, start_year, end_year))
